In [5]:
import numpy as np
import os
import json
import sys
import torch
import math
import time

from scipy.spatial.transform import Rotation as R

In [28]:
def read_input(file_path):
    with open(file_path) as json_data:
        d = json.load(json_data)

    return d['positions'], d['rotations'], d['parents'], d['foot_contact']

def main():
    # reading all data
    file_path = "C:\\Users\\eggyr\\OneDrive\\RPI\\S10\\Projects in ML\\final\\lafan1_detail_model_benchmark_5_0-2231.json"
    all_orig_positions, all_orig_rotations, parents, all_foot_contact = read_input(file_path)
    all_orig_positions = np.array(all_orig_positions)
    all_orig_rotations = np.array(all_orig_rotations)
    new_joints = 10
    all_new_positions = np.array((all_orig_positions.shape[0], all_orig_positions.shape[1], new_joints, 3))
    all_new_rotations = np.array((all_orig_rotations.shape[0], all_orig_rotations.shape[1], new_joints, 3))
    print(f'original positions shape: {all_orig_positions.shape}')

    all_new_positions, all_new_rotations = conv_rig(all_orig_positions, all_orig_rotations, parents)
    #print(f"all pos: {all_new_positions}\nall rot: {all_new_rotations}")
    return
    for sequence in range(all_orig_positions.shape[0]):
        all_new_positions[sequence], all_new_rotations[sequence] = conv_rig(all_orig_positions[sequence], all_orig_rotations[sequence], parents)
        print(f"all pos: {all_new_positions}\nall rot: {all_new_rotations}")
        return


def conv_rig(positions, rotations, parents):
    # takes in original positions and rotations and returns the new representation of positions and rotations
    global_rot, global_pos = fk_torch(torch.from_numpy(rotations), torch.from_numpy(positions), parents)
    global_pos = global_pos.numpy()
    global_rot = global_rot.numpy()
    print(f"all pos {global_pos.shape}: {global_pos}\nall rot {global_rot.shape}: {global_rot}")
    return global_pos, global_rot

    
def fk_torch(lrot, lpos, parents):
    """
    Calculate forward kinematics.

    Args:
        lrot (Tensor): Local rotation of joints. Shape: (..., joints, 3, 3)
        lpos (Tensor): Local position of joints. Shape: (..., joints, 3)
        parents (list of int or 1D int Tensor): Parent indices.

    Returns:
        Tensor, Tensor: (global rotation, global position).
            Shape: (..., joints, 3, 3), (..., joints, 3)
    """
    gr = [lrot[..., :1, :, :]]
    gp = [lpos[..., :1, :]]

    print(f'{gr[0].shape}, {gp[0].shape}')

    for i in range(1, len(parents)):
        gr_parent = gr[parents[i]]
        gp_parent = gp[parents[i]]

        gr_i = torch.matmul(gr_parent, lrot[..., i:i + 1, :, :])
        gp_i = gp_parent + \
            torch.matmul(gr_parent, lpos[..., i:i + 1, :, None]).squeeze(-1)

        gr.append(gr_i)
        gp.append(gp_i)

    return torch.cat(gr, dim=-3), torch.cat(gp, dim=-2)

In [ ]:
def create_object(position, id):
    bpy.ops.object.empty_add(type='PLAIN_AXES', location=position)

    # store the created Empty in a variable for further manipulation
    new_empty = bpy.context.object
    new_empty.name = f"{id}"  # Rename the Empty
    #rotate_about_world_origin(new_empty, (3,0,0))
    return new_empty
    
    
def rotate_about_world_origin(x, rotation):
    rot_mat = Euler(rotation).to_matrix().to_4x4()
    x.matrix_world = rot_mat @ x.matrix_world
    

def move_bone(bone_name, amount):
    # Enter pose mode so rig can be modified
    bpy.ops.object.mode_set(mode='POSE')

    # Specify the armature object
    armature = bpy.data.objects['rig1']

    # Specify the bone
    bone = armature.pose.bones[bone_name]

    # Set the bone's rotation mode to Euler XYZ, if not already
    bone.rotation_mode = 'XYZ'
    bone.location[2] += amount[0]
    bone.location[1] += amount[1]
    bone.location[0] += amount[2]


def test_movement(iterations):
    times = np.zeros((iterations,))
    
    for i in range(iterations):

        move_bone('footcontrol.l', [0, -0.5, 0.5])

        start_time = time.time()
        bpy.context.view_layer.update()
        end_time = time.time()
        ik_time = end_time - start_time

        move_bone('footcontrol.l', [0, 0.5, -0.5])
        bpy.context.view_layer.update()
        
        times[i] = ik_time
        
    return times


#times = test_movement(1000)
#print(f'average solve time: {np.mean(times)*1000} ms, std {np.std(times)*1000} ms')




def visualize(json_path, joint_r=5, foot_joint_idx=(3, 4, 7, 8)):
    baseName = os.path.basename(json_path).rsplit(".", 1)[0]

    with open(json_path) as fh:
        d = json.load(fh)

    print(f'{d.keys()}')

    points_list = d["positions"]
    points_list = np.array(points_list)
    print(f'points shape: {points_list.shape}')
    
    for frame in range(points_list.shape[0]):
        for joint in range(points_list.shape[1]):
            create_object(points_list[frame, joint], f"{frame}.{joint}")
        return
    return


def read_input(file_path):
    with open(file_path) as json_data:
        d = json.load(json_data)

    return d['positions'], d['rotations'], d['parents'], d['foot_contact']

def init_blend():
    for i in range(22):
        create_object((0,0,0), f"joint.{i}")

def main(file_path):
    # reading all data
    init_blend()
    all_orig_positions, all_orig_rotations, parents, all_foot_contact = read_input(file_path)
    all_orig_positions = np.array(all_orig_positions)
    all_orig_rotations = np.array(all_orig_rotations)
    new_joints = 10
    all_new_positions = np.array((all_orig_positions.shape[0], all_orig_positions.shape[1], new_joints, 3))
    all_new_rotations = np.array((all_orig_rotations.shape[0], all_orig_rotations.shape[1], new_joints, 3))
    print(f'original positions shape: {all_orig_positions.shape}')

    all_new_positions, all_new_rotations = conv_rig(all_orig_positions, all_orig_rotations, parents)
    #print(f"all pos: {all_new_positions}\nall rot: {all_new_rotations}")
    for sequence in range(all_orig_positions.shape[0]):
        if sequence != 0:
            continue
        for frame in range(all_orig_positions.shape[1]):
            for joint in range(all_orig_positions.shape[2]):
                x = bpy.data.objects[f"joint.{joint}"]
                x.location = all_new_positions[sequence, frame, joint]
                bpy.context.view_layer.update()
                rotate_about_world_origin(x, (pi/2,0,0))
                bpy.context.view_layer.update()
                x.keyframe_insert(data_path="location", frame=frame)
        return

def conv_rig(positions, rotations, parents):
    # takes in original positions and rotations and returns the new representation of positions and rotations
    global_rot, global_pos = fk(rotations, positions, parents)
    print(f"all pos {global_pos.shape}: {global_pos}\nall rot {global_rot.shape}: {global_rot}")
    return global_pos, global_rot

    
def fk(lrot, lpos, parents):
    """
    Calculate forward kinematics.

    Args:
        lrot (Tensor): Local rotation of joints. Shape: (..., joints, 3, 3)
        lpos (Tensor): Local position of joints. Shape: (..., joints, 3)
        parents (list of int or 1D int Tensor): Parent indices.

    Returns:
        Tensor, Tensor: (global rotation, global position).
            Shape: (..., joints, 3, 3), (..., joints, 3)
    """
    gr = [lrot[..., :1, :, :]]
    gp = [lpos[..., :1, :]]

    print(f'{gr[0].shape}, {gp[0].shape}')

    for i in range(1, len(parents)):
        gr_parent = gr[parents[i]]
        gp_parent = gp[parents[i]]

        gr_i = np.matmul(gr_parent, lrot[..., i:i + 1, :, :])
        gp_i = gp_parent + \
            np.matmul(gr_parent, lpos[..., i:i + 1, :, None]).squeeze(-1)

        gr.append(gr_i)
        gp.append(gp_i)

    return np.concatenate(gr, axis=-3), np.concatenate(gp, axis=-2)


#create_object((1,0,0), 'test')
main("C:\\Users\\eggyr\\OneDrive\\RPI\\S10\\Projects in ML\\final\\lafan1_detail_model_benchmark_5_0-2231.json")


In [ ]:
def rotate_point(point, axis, angle, pivot):
    # Convert angle from degrees to radians
    angle_rad = np.radians(angle)
    
    # Create a rotation object using the axis and angle
    rotation = R.from_rotvec(angle_rad * np.array(axis))
    
    # Get the rotation matrix
    rot_matrix = rotation.as_matrix()
    
    # Translate point to origin by subtracting pivot
    point_translated = np.array(point) - np.array(pivot)
    
    # Apply rotation
    rotated_point_translated = np.dot(rot_matrix, point_translated)
    
    # Translate back to the original location
    final_point = rotated_point_translated + np.array(pivot)
    
    return final_point

# Example usage
point = [1, 0, 0]  # Point to be rotated
axis = [0, 0, 1]   # Axis of rotation (z-axis)
angle = 90         # Rotation angle in degrees
pivot = [0, 0, 0]  # Pivot point for rotation

rotated_point = rotate_point(point, axis, angle, pivot)
print("Rotated Point:", rotated_point)


In [29]:
main()

original positions shape: (2232, 17, 22, 3)
torch.Size([2232, 17, 1, 3, 3]), torch.Size([2232, 17, 1, 3])
all pos (2232, 17, 22, 3): [[[[-1.56081840e-01  9.24606934e+01  1.56027332e-01]
   [ 1.73440897e+00  9.25397711e+01 -1.03868818e+01]
   [ 1.15063179e+00  4.90653653e+01 -1.17601906e+01]
   ...
   [ 4.62432365e-01  1.44632054e+02  1.77451277e+01]
   [-3.39220474e+00  1.46821267e+02  5.04461180e+01]
   [-9.98379223e-01  1.51668342e+02  7.50592152e+01]]

  [[-1.35523871e-01  9.24608765e+01  1.43355191e-01]
   [ 1.75496222e+00  9.25401472e+01 -1.03995533e+01]
   [ 1.15269652e+00  4.90659966e+01 -1.17729023e+01]
   ...
   [ 4.72152305e-01  1.44631520e+02  1.77328894e+01]
   [-3.37852520e+00  1.46810728e+02  5.04350143e+01]
   [-9.66868797e-01  1.51656002e+02  7.50467249e+01]]

  [[-1.04587667e-01  9.24597321e+01  1.25545427e-01]
   [ 1.78585797e+00  9.25394226e+01 -1.04173671e+01]
   [ 1.15801415e+00  4.90655593e+01 -1.17884063e+01]
   ...
   [ 4.85513976e-01  1.44629168e+02  1.77156250